In [1]:
import numpy as np
from scipy import special as sp
import matplotlib.pyplot as plt
import sys
if "../src/" not in sys.path:
    sys.path.insert(0,"../src/")
from pbcpy.grid import DirectGrid, ReciprocalGrid
from pbcpy.field import DirectField, ReciprocalField
from pbcpy.functionals import *
from pbcpy.local_functionals_utils import TF
from pbcpy.semilocal_xc import LDA,PBE
from pbcpy.local_pseudopotential import NuclearElectron
from pbcpy.hartree import HartreeFunctional, HartreePotentialReciprocalSpace
%matplotlib inline
%load_ext autoreload
%autoreload 2
#
from pbcpy.formats.qepp import PP
from pbcpy.formats.xsf import XSF

# Starting Ewald

In [2]:
def Get_Gmax(grid):
    gg = grid.get_reciprocal().gg
    gmax_x = np.sqrt(np.amax(gg[:,0,0]))
    gmax_y = np.sqrt(np.amax(gg[0,:,0]))
    gmax_z = np.sqrt(np.amax(gg[0,0,:]))
    gmax = np.amin([gmax_x,gmax_y,gmax_z])
    return gmax

In [3]:
def Get_Best_eta(precision,gmax,ions):
    '''
    INPUT: precision, gmax & ions
    OUTPUT: eta
    '''
    
    # charge
    charge = 0.0
    for i in np.arange(len(ions)):
        charge+=ions[i].Zval
    
    #eta
    eta = 1.2
    NotGoodEta = True
    while NotGoodEta:
        upbound = 2.0 * charge**2 * np.sqrt ( eta / np.pi) * sp.erfc ( np.sqrt (gmax / 4.0 / eta) )
        if upbound<precision:
            NotGoodEta = False
        else:
            eta = eta - 0.1
    return eta

# Total Energy

In [4]:
def TotalEnergy(precision,ions,rho):
    KEDF = KineticEnergy(rho)
    XC = ExchangeCorrelationEnergy(rho)
    Hartree = HartreeFunctional(rho).energydensity.integral()
    gmax = Get_Gmax(rho.grid)
    eta = Get_Best_eta(precision, gmax, ions)
    DivergentTerm = Ediv1(ions,rho)+Ediv2(precision,eta,ions,rho)
    print('Kinetic = ', KEDF)
    print('     XC = ', XC)
    print('Hartree = ', Hartree)
    print('Ewald = ', DivergentTerm)
    print('eta used = ', eta)
    return KEDF+XC+DivergentTerm

In [5]:
def KineticEnergy(rho):
    return TF(rho).energydensity.integral()

In [6]:
def ExchangeCorrelationEnergy(rho):
    return PBE(rho,polarization='unpolarized').energydensity.integral()

In [7]:
def Eewald1(eta, charges, positions):
    Esum=np.float(0.0)
    for i in range(len(charges)):
        for j in range(len(charges)):
            if i!=j:
                rij=positions[i]-positions[j]
                dij=rij.length()
                Esum+=charges[i]*charges[j]*sp.erfc(np.sqrt(eta)*dij)/dij
    return Esum/2.0

In [8]:
def Eewald2(eta,ions,rho):
    
    #rec space sum
    reciprocal_grid = rho.grid.get_reciprocal()
    gg=reciprocal_grid.gg
    strf = ions[0].strf(reciprocal_grid) * ions[0].Zval
    for i in np.arange(1,len(ions)):
        strf += ions[i].strf(reciprocal_grid) * ions[i].Zval
    strf_sq =np.conjugate(strf)*strf
    gg[0,0,0,0]=1.0
    invgg=1.0/gg
    invgg[0,0,0,0]=0.0
    gg[0,0,0,0]=0.0
    First_Sum=np.real(4.0*np.pi*np.sum(strf_sq*np.exp(-gg/(4.0*eta))*invgg)) / 2.0 / rho.grid.volume
    
    # double counting term
    const=-np.sqrt(eta/np.pi)
    sum = np.float(0.0)
    for i in np.arange(len(ions)):
        sum+=ions[i].Zval**2
    dc_term=const*sum 
    
    # G=0 term of local_PP - Hartree
    const=-4.0*np.pi*(1.0/(4.0*eta*rho.grid.volume)/2.0)
    sum = np.float(0.0)
    for i in np.arange(mol.natoms):
        sum+=ions[i].Zval
    gzero_limit=const*sum**2
    
    return First_Sum+dc_term+gzero_limit


In [9]:
def Ediv2(precision,eta,ions,rho):
    L=np.sqrt(np.einsum('ij->j',rho.grid.lattice**2))
    prec = sp.erfcinv(precision/3.0)
    rmax = np.array([ prec / np.sqrt(eta), prec / np.sqrt(eta), prec / np.sqrt(eta)])
    N=np.rint(rmax/L)
    print('Map of Cells = ',N)
    print('Lengths = ',rmax/L)
    charges = []
    positions = []
    sum = np.float(0.0)
    for ix in np.arange(-N[0]+1,N[0]):
        for iy in np.arange(-N[1]+1,N[1]):
            for iz in np.arange(-N[2]+1,N[2]):
                R=np.einsum('j,ij->i',np.array([ix,iy,iz],dtype=np.float),rho.grid.lattice.transpose())
                for i in np.arange(len(ions)):
                    charges.append(ions[i].Zval)
                    positions.append(ions[i].pos-R)
    output = Eewald1(eta,charges,positions)+Eewald2(eta,ions,rho)
    output = output-np.sum(np.real(HartreePotentialReciprocalSpace(density=rho)*np.conjugate(rho.fft())))/2.0/rho.grid.volume
    return output

In [10]:
def Ediv1(ions,rho):
    
    # alpha Z term:
    alpha = 0.0
    for i in range(len(ions)):
        alpha += ions[i].alpha_mu
    Z = 0.0
    for i in range(len(ions)):
        Z +=ions[i].Zval
    alpha_z = alpha * Z / rho.grid.volume
    
    # twice Hartree term
    rhog = rho.fft()
    TwoEhart = np.sum(np.real(HartreePotentialReciprocalSpace(density=rho)*np.conjugate(rhog)))/rho.grid.volume
    vloc = np.zeros_like(rhog)
    for i in range(len(ions)):
        vloc+=ions[i].v
    vloc[0,0,0,0] = 0.0
    Eloc = np.real(np.sum(np.conj(rhog)*vloc))/rho.grid.volume
    return alpha_z+TwoEhart+Eloc

# Test

# get a test density and grid

In [11]:
mol = PP(filepp='Al_fde_rho.pp').read()
rho_of_r = mol.field

# load PPs
Nucel = NuclearElectron(mol.ions,rho_of_r,['./Al_lda.oe01.recpot','./Al_lda.oe01.recpot'])

PP_file not set in input. Can do so manually invoking Atom.local_PP
PP_file not set in input. Can do so manually invoking Atom.local_PP
Recpot pseudopotential ./Al_lda.oe01.recpot loaded
Recpot pseudopotential ./Al_lda.oe01.recpot loaded


In [12]:
TotalEnergy(precision=1.0e-9,ions=mol.ions,rho=rho_of_r)*27.211/2

Map of Cells =  [ 2.  2.  1.]
Lengths =  [ 1.83733153  1.83733153  0.91866576]
Kinetic =  1.61747277437
     XC =  -1.63441442372
Hartree =  0.00839883264471
Ewald =  -4.08563080666
eta used =  0.20000000000000004


-55.817549550334199

# now `ewald` is a class of its own

In [58]:
from pbcpy.ewald import ewald

In [59]:
ewald = ewald(ions=mol.ions,precision=1.0e-9,rho=rho_of_r)

In [60]:
ewald.energy

-4.0856308066616229